In [83]:
# Train path 
TRAIN_PATH = "CovidDataset/train"

In [84]:
# Val path
VAL_PATH = "CovidDataset/test"

In [85]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
import tensorflow as tf
import os

In [86]:
assert tf.__version__.startswith('2')

from keras.preprocessing import *


# CNN Based Model in Keras

In [87]:

model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape = (224,224,3)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', ))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])

In [88]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 110, 110, 64)      0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 54, 54, 64)       

## Train the Model

In [89]:
# Train from Scratch

train_datagen = image.ImageDataGenerator(
    rescale = 1/255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
)

test_dataset  = image.ImageDataGenerator(rescale=1./255)


train_generator  = train_datagen.flow_from_directory(
    'CovidDataset/train',
    target_size = (224,224),
   batch_size = 50,
    class_mode = 'binary'
)
#as 4390 images for setting test batch size=50 and number of steps 88 so it go through
#test set once per epoch 50x88=4400

Found 4393 images belonging to 2 classes.


In [90]:
train_generator.class_indices


{'Negative': 0, 'Positive': 1}

In [91]:

validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/test',
    target_size = (224,224),
    batch_size = 50,
    class_mode = 'binary'

)
#as 500 images for setting validation batch size=50 and number of steps 10 so it go through
#validation set once per epoch 50x10=500

Found 500 images belonging to 2 classes.


In [92]:
print (train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)

{'Negative': 0, 'Positive': 1}


In [93]:
#!cat labels.txt

In [94]:
validation_generator.class_indices


{'Negative': 0, 'Positive': 1}

In [95]:
# number of training images we have are

train_generator.n

4393

In [96]:
# number of test images we have are  

validation_generator.n

500

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch = 88,
    #as 4390 images for setting test batch size=50 and number of steps 88 so it go through
    #test set once per epoch 50x88=4400
    epochs =10,
    validation_data = validation_generator,
    validation_steps = 10
    #as 500 images for setting validation batch size=50 and number of steps 10 so it go through
    #validation set once per epoch 50x10=500
)

Epoch 1/10
88/88 [==============================] - 1868s 21s/step - loss: 0.5689 - accuracy: 0.7938 - val_loss: 0.2571 - val_accuracy: 0.9760
Epoch 2/10
88/88 [==============================] - 1689s 19s/step - loss: 0.3270 - accuracy: 0.8919 - val_loss: 0.2233 - val_accuracy: 0.9700- ETA: 20:21 - loss: 0.3154 - accur - ETA: 15:0 - ET
Epoch 3/10
88/88 [==============================] - 1898s 22s/step - loss: 0.2982 - accuracy: 0.8992 - val_loss: 0.1428 - val_accuracy: 0.9700
Epoch 4/10
88/88 [==============================] - 1331s 15s/step - loss: 0.2947 - accuracy: 0.9019 - val_loss: 0.1969 - val_accuracy: 0.9800: 4:30 - l - ETA: 58s - loss: 0.2964 - accuracy: 0.9
Epoch 5/10
88/88 [==============================] - 1813s 21s/step - loss: 0.2803 - accuracy: 0.9080 - val_loss: 0.1514 - val_accuracy: 0.9700
Epoch 6/10
88/88 [==============================] - 1795s 20s/step - loss: 0.2688 - accuracy: 0.9087 - val_loss: 0.2124 - val_accuracy: 0.9660TA: 14:37 - loss: 0.2683 - accuracy: 0 

In [ ]:
model.save('model_advance.h5')

In [ ]:
saved_model_dir = 'save/fine_tuning'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('model1.tflite', 'wb') as f:
  f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: save/fine_tuning/assets


In [ ]:
from google.colab import files

files.download('model1.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model.evaluate_generator(train_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


Instructions for updating:
Please use Model.evaluate, which supports generators.


[0.09385678917169571, 0.9732142686843872]